In [15]:
import pandas as pd 
import os 
import sqlite3 
import shutil
import glob
import numpy as np
from datetime import date
import datetime
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


production = sqlite3.connect('C:\\python_development\\productivity.db', timeout=30) 
#pd.read_sql_query('SELECT * FROM sqlite_master where name like "%vw%";', production)

In [27]:
# file path definition
# Set the paths for import and export files
import_path = r"\\vault\powerbiflatfiles$\Production\MGMA Benchmarks\\"
#export_folder = r"\\vault\powerbiflatfiles$\Production\Data_Model\file_transformation\export_files\\"

xls_files = []

xls_files.extend(glob.glob(import_path + "*.XLSX"))


In [19]:
# loop through files
if not xls_files:
    print("No .xls files found in the specified directory.")
else:
    print("Convert each .xls file to .xlsx and save to the export folder")
    for xls_file in xls_files:
        try:
            # Read the .xls file
            if 'Payment MGMADataDive' in xls_file:
                df = pd.read_excel(xls_file, engine="xlrd") 

                # take the X report, add 1 and set that as fiscal year
                print(xls_file)
                break

                # add the index for specialty id from established source

                # clean the data
                columns_to_clean = ['Mean', '10th %tile', '25th %tile', 'Median', '75th %tile', '90th %tile']
                for column in columns_to_clean:
                    df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)
                df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')
                df = df.dropna(subset=['Mean'])
                for column in columns_to_clean:
                df[column] = df[column].astype(int)

            if 'wRVU MGMADataDive' in xls_file:
                df = pd.read_excel(xls_file, engine="xlrd")
                

                # Define the new .xlsx file path
                
              
        except Exception as e:
            print(f"Error converting {xls_file}: {e}") 

<>:21: SyntaxWarning: invalid escape sequence '\$'
<>:21: SyntaxWarning: invalid escape sequence '\*'
<>:21: SyntaxWarning: invalid escape sequence '\$'
<>:21: SyntaxWarning: invalid escape sequence '\*'
C:\Users\dsarmien\AppData\Local\Temp\ipykernel_1884\1845585345.py:21: SyntaxWarning: invalid escape sequence '\$'
  df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)
C:\Users\dsarmien\AppData\Local\Temp\ipykernel_1884\1845585345.py:21: SyntaxWarning: invalid escape sequence '\*'
  df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)


IndentationError: expected an indented block after 'for' statement on line 24 (1845585345.py, line 25)

In [33]:

# Loop through files
if not xls_files:
    print("No .xlsx files found in the specified directory.")
else:
    print("Import each excel file to benchmarks")

    for xls_file in xls_files:
        try:
            # Read the .xls file
            df = pd.read_excel(xls_file, engine="openpyxl")

            if 'Payment MGMADataDive' in xls_file:
                print(f"Processing: {xls_file}")
                break

                # Clean the data
                columns_to_clean = ['Mean', '10th %tile', '25th %tile', 'Median', '75th %tile', '90th %tile']
                for column in columns_to_clean:
                    df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)
                
                df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')
                df = df.dropna(subset=['Mean'])

                for column in columns_to_clean:
                    df[column] = df[column].astype(int)

            elif 'wRVU MGMADataDive' in xls_file:
                print(f"Processing: {xls_file}")

            # Define the new .xlsx file path (add export folder handling)
            export_path = os.path.join("export_folder", os.path.splitext(os.path.basename(xls_file))[0] + ".xlsx")
            #df.to_excel(export_path, index=False, engine="openpyxl")


        except Exception as e:
            print(f"Error converting {xls_file}: {e}")


Import each excel file to benchmarks
Processing: \\vault\powerbiflatfiles$\Production\MGMA Benchmarks\Payment MGMADataDive 2018 Report based on 2017 Data.xlsx


<>:19: SyntaxWarning: invalid escape sequence '\$'
<>:19: SyntaxWarning: invalid escape sequence '\*'
<>:19: SyntaxWarning: invalid escape sequence '\$'
<>:19: SyntaxWarning: invalid escape sequence '\*'
C:\Users\dsarmien\AppData\Local\Temp\ipykernel_1884\1510843144.py:19: SyntaxWarning: invalid escape sequence '\$'
  df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)
C:\Users\dsarmien\AppData\Local\Temp\ipykernel_1884\1510843144.py:19: SyntaxWarning: invalid escape sequence '\*'
  df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)


In [ ]:
# Payment MGMA update
# Importing new benchmarks is a multi step process: 1) import and clean source data file, 
# 2) create an insert file to add specialty_id to new rows, 3) insert into src_mgma_payment_bench_all 

# Update this file name when new payment report available
df = pd.read_excel('V:\\Production\\MGMA Benchmarks\\Payment MGMADataDive 2024 Report based on 2023 Data.xlsx', header=13)
# List of columns to clean
columns_to_clean = ['Mean', '10th %tile', '25th %tile', 'Median', '75th %tile', '90th %tile']

# Loop through each column and remove the dollar signs, commas, and asterisks
for column in columns_to_clean:
    df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)
    
# If you also want to convert them to numeric values (floats or integers), you can add:
df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')
df = df.dropna(subset=['Mean'])

for column in columns_to_clean:
    df[column] = df[column].astype(int)

df.columns = ['specialty','mean','10th_ntile','25th_ntile','median','75th_ntile','90th_ntile']
df.to_sql('temp_mgma_payments', production, if_exists='replace', index=False)
df.head()

In [ ]:
query = '''
select
d.specialty_id
, t.*
, '2025' as fy
from temp_mgma_payments as t
inner join dim_mgma_specialty as d on d.Specialty = t.specialty
'''
df = pd.read_sql_query(query, production)
df.head()

In [ ]:
# append rows to sqlite 
df.to_sql('src_mgma_payment_bench_all', production, if_exists='append', index=False)
# write updated benchmarking file
df.to_csv('V:\\Production\\MGMA Benchmarks\\src_mgma_payment_bench_all.csv')

In [8]:
# RVU MGMA source import
# Importing new benchmarks is a multi step process: 1) import and clean source data file, 
# 2) create an insert file to add specialty_id to new rows, 3) insert into src_mgma_rvu_bench_all 

# Update this file name when new rvu report available
df = pd.read_excel('V:\\Production\\MGMA Benchmarks\\wRVU MGMADataDive 2024 Report based on 2023 Data.xlsx', header=13)
# List of columns to clean
columns_to_clean = ['Mean', '10th %tile', '25th %tile', 'Median', '75th %tile', '90th %tile']

# Loop through each column and remove the dollar signs, commas, and asterisks
for column in columns_to_clean:
    df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)
    
# If you also want to convert them to numeric values (floats or integers), you can add:
df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')
df = df.dropna(subset=['Mean'])

for column in columns_to_clean:
    df[column] = df[column].astype(int)

df.columns = ['specialty','mean','10th_ntile','25th_ntile','median','75th_ntile','90th_ntile']
df.to_sql('temp_mgma_rvu', production, if_exists='replace', index=False)
df.head()

<>:12: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\*'
<>:12: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\*'
C:\Users\dsarmien\AppData\Local\Temp\ipykernel_10668\1216408702.py:12: SyntaxWarning: invalid escape sequence '\$'
  df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)
C:\Users\dsarmien\AppData\Local\Temp\ipykernel_10668\1216408702.py:12: SyntaxWarning: invalid escape sequence '\*'
  df[column] = df[column].replace({'\$': '', ',': '', '\*': ''}, regex=True)


,specialty,mean,10th_ntile,25th_ntile,median,75th_ntile,90th_ntile
0,Allergy/Immunology,5300,2446,3398,4907,6661,8650
1,Anesthesiology: Pain Management,8258,3355,5677,7986,10550,13469
2,Bariatrics (Nonsurgical)/Obesity Medicine,4946,1991,3151,5339,6619,7013
3,Cardiology: Electrophysiology,12701,6291,9159,12257,15691,20275
4,Cardiology: Invasive,11044,6246,8282,10738,13171,15838


In [24]:
# insert file
query = '''
select
d.specialty_id
, t.*
, '2025' as fy
from temp_mgma_rvu as t
inner join dim_mgma_specialty as d on d.Specialty = t.specialty
'''
df = pd.read_sql_query(query, production)
df.head()

,specialty_id,specialty,mean,10th_ntile,25th_ntile,median,75th_ntile,90th_ntile,fy
0,1001,Allergy/Immunology,5300,2446,3398,4907,6661,8650,2025
1,1002,Anesthesiology: Pain Management,8258,3355,5677,7986,10550,13469,2025
2,1003,Bariatrics (Nonsurgical)/Obesity Medicine,4946,1991,3151,5339,6619,7013,2025
3,1004,Cardiology: Electrophysiology,12701,6291,9159,12257,15691,20275,2025
4,1005,Cardiology: Invasive,11044,6246,8282,10738,13171,15838,2025


In [26]:
# append rows to sqlite 
df.to_sql('src_mgma_rvu_bench_all', production, if_exists='append', index=False)
# write to benchmarks folder
df.to_csv('V:\\Production\\MGMA Benchmarks\\src_mgma_rvu_bench_all.csv')

205